In [1]:
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import urllib.request
import numpy as np
from datetime import datetime
# print message after packages imported successfully
print("import of packages successful")
##import of packages successful

import of packages successful


In [2]:
#Data Path
os.chdir("/home/mohamedali/Desktop/Matrix_factorization/nf_prize_dataset/download")

# Methodology

## 1. Data Compilation

### 1.1. Generation of the rating dataset

* Write Python function **rating_compiler** to compile from the training_set.tar, the $17770$ files and store the result into one data structure named $\bf{D}$;

* $\bf{D}$ can be a numpy matrix, csv file/pandas DataFrame, sparse matrix, hdf5 file...;

In [41]:
#Sparse Matrix
##Initializing the Data frame
dt=np.dtype({'names': ('MovieID','Rating','Date'),'formats': ('int16', 'int8', 'datetime64[D]')})
D = np.zeros((2649429,4), dtype=dt)
##Data frame size
print("%d Megabytes" % ((D.size * D.itemsize)/1000000))

116 Megabytes


In [5]:
def rating_compiler(previous=None):
    for i in range(1, 11):
        file = "training_set" + "/mv_00" + '{0:05}'.format(i) + '.txt'
        dt=np.dtype({'names': ('CustomerID','Rating','Date'),'formats': ('int32', 'int8', 'datetime64[D]')})
        data = np.loadtxt(fname = file,skiprows=1, delimiter=',', dtype=dt)
        new_data = np.array(data.tolist())
        leng = np.shape(data)[0]
        movie_id = np.full((leng,1),i)
        new_data = np.append(new_data, movie_id, axis=1)
        print(i)
        if(i==1):
            previous = new_data
        else:
            previous = np.concatenate((previous, new_data))

    return(previous)

In [6]:
D = rating_compiler()

1
2
3
4
5
6
7
8
9
10


array([[1488844, 3, datetime.date(2005, 9, 6), 1],
       [822109, 5, datetime.date(2005, 5, 13), 1],
       [885013, 4, datetime.date(2005, 10, 19), 1],
       ...,
       [1796454, 3, datetime.date(2004, 1, 29), 10],
       [1213178, 3, datetime.date(2005, 12, 10), 10],
       [515617, 4, datetime.date(2003, 3, 28), 10]], dtype=object)

### 1.2. Generation of training and test datasets

* Extract from $\bf{D}$ the ratings corresponding to the users and movies described in the probe.txt and store the result into one data structure named $\bf{T}$;

In [ ]:
#R=D-T
#T=D.ratings==probe.txt

In [50]:
#Extract the character before :
def substring_after(s, delimiter):
    return s.partition(delimiter)[0]

In [48]:
def getting_t():
    the_array = np.array([])
    with open('t.txt') as f:
        customer_id = 0
        movie_ids = 0
        for line in f:
           # For Python3, use print(line)
           #Compare if it is a Movie or a Customer Id
           # #stoke the movie id
            if ':' in line:
                movie_ids = int(substring_after(line,":"))
                print("Movie id",movie_ids)

            elif ":" not in line:
                customer_id = int(line)
                print("Customer_id : ",customer_id)

                if the_array.size == 0:
                    the_array= np.select(((D[:,0]==customer_id) * (D[:,3]==movie_ids)),D).reshape((1,4))
                else:
                    the_array = np.concatenate((the_array,np.select(((D[:,0]==customer_id) * (D[:,3]==movie_ids)),D).reshape((1,4))))


            if 'str' in line:
                break
    print("Finish")
    return the_array

In [58]:
T = getting_t()

Movie id 1
Customer_id :  30878
Customer_id :  2647871
Customer_id :  1283744
Customer_id :  2488120
Customer_id :  317050
Customer_id :  1904905
Customer_id :  1989766
Customer_id :  14756
Customer_id :  1027056
Customer_id :  1149588
Customer_id :  1394012
Customer_id :  1406595
Customer_id :  2529547
Customer_id :  1682104
Customer_id :  2625019
Customer_id :  2603381
Customer_id :  1774623
Customer_id :  470861
Customer_id :  712610
Customer_id :  1772839
Customer_id :  1059319
Customer_id :  2380848
Customer_id :  548064
Movie id 10
Customer_id :  1952305
Customer_id :  1531863
Finish


* Construct training dataset $\bf{R}$ as $\bf{D}$ from which we remove entries present in $\bf{T}$;

* $\bf{D}$, $\bf{T}$ and $\bf{R}$ must all have the same format (for example users as rows and films as columns);

In [64]:
D_pd = pd.DataFrame({'Customer_id': D[:, 0], 'Rating': D[:, 1], 'Date': D[:, 2], 'Movie_id': D[:, 3]})
T_pd = pd.DataFrame({'Customer_id': T[:, 0], 'Rating': T[:, 1], 'Date': T[:, 2], 'Movie_id': T[:, 3]})

      Customer_id Rating        Date Movie_id
0         1488844      3  2005-09-06        1
1          822109      5  2005-05-13        1
2          885013      4  2005-10-19        1
3           30878      4  2005-12-26        1
4          823519      3  2004-05-03        1
...           ...    ...         ...      ...
20347       88188      3  2005-12-06       10
20348     1462129      4  2005-03-11       10
20349     1796454      3  2004-01-29       10
20350     1213178      3  2005-12-10       10
20351      515617      4  2003-03-28       10

[20352 rows x 4 columns]


In [71]:
df1 = D_pd
df2 = T_pd

In [74]:
#First Method
R_pd1= df1[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]

In [68]:
#Second Method
R_pd2 = T_pd.merge(D_pd, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']